# Phase 0→1: Document Structure Re-extraction

**What this notebook does**: Re-walks the EPUB using the CSS class map discovered in Phase 0, building a proper document hierarchy with:
- ✓ Real chapter titles (not null)
- ✓ Section/subsection heading hierarchy (h1→h2→h3→h4)
- ✓ Structural role for every paragraph (body, verse, instruction, etc.)
- ✓ Paragraph text preserved exactly as published

**What it replaces**: `06_document_structure_layer1.json` (which had 87/88 null chapter titles)

**What it does NOT touch**: Phase 2 NLP pipeline, vocabulary, or ChromaDB vectors.

In [1]:
import os
import json
import re
from collections import defaultdict, OrderedDict
from dataclasses import dataclass, asdict, field
from typing import Optional, List

import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

# ============================================================
# CONFIGURATION
# ============================================================
EPUB_DIR = os.path.expanduser("~/Documents/gesha_la_rag/epub_directory/")
EPUB_FILE = "Clear_Light_of_Bliss.epub"
EPUB_PATH = os.path.join(EPUB_DIR, EPUB_FILE)
OUTPUT_FILE = "06_document_structure_layer1.json"

print(f"EPUB: {EPUB_PATH}")
print(f"Exists: {os.path.exists(EPUB_PATH)}")

EPUB: /home/matt/Documents/gesha_la_rag/epub_directory/Clear_Light_of_Bliss.epub
Exists: True


## CSS Class → Structural Role Map

Built from Phase 0 inventory of 121 CSS classes across 89 EPUB sections. Every `<p>` tag class is mapped to a structural role and heading level.

**Heading hierarchy** (as Tharpa uses it):
- **Level 1** = Chapter titles (`Chapter-title-TOC-Level-1`)  
- **Level 2** = Section headings within chapters (`Text-heading-1`, `Chapter-title-TOC-Level-2`, `Sadhana-heading-2`)
- **Level 3** = Sub-section headings (`Sadhana-heading-3`, `Text-heading-3-italics-brackets`)
- **Level 4** = Sub-sub-section headings (`Sadhana-heading-4`)

Note: `Text-heading-1` is confusingly named — it's actually h2 (section within chapter), not h1. The `1` refers to Tharpa's internal style numbering, not document hierarchy.

In [2]:
# ============================================================
# COMPLETE CSS CLASS → STRUCTURAL ROLE MAP
# ============================================================
# Built from Phase 0 inventory (121 classes, 89 sections)
#
# Each entry: 'CSS-Class-Name': ('ROLE', heading_level_or_None)
#
# Roles:
#   CHAPTER_TITLE  - Top-level chapter heading (h1)
#   SECTION_HEAD   - Section heading within chapter (h2)
#   SUBSECTION     - Sub-section heading (h3)
#   SUBSUBSECTION  - Sub-sub-section heading (h4)
#   BODY           - Regular body paragraph
#   BODY_FIRST     - First paragraph after a heading (same structural level, different styling)
#   VERSE          - Verse/poetry quotation
#   QUOTE_PROSE    - Prose quotation
#   LIST_ITEM      - Numbered/bulleted list entry
#   INSTRUCTION    - Sadhana/meditation instruction
#   SADHANA_PROSE  - Sadhana liturgical text
#   MANTRA         - Mantra text
#   CHART_TITLE    - Chart/table title
#   IMAGE_CAPTION  - Image caption
#   GLOSSARY       - Glossary entry
#   INDEX          - Index entry
#   TOC            - Table of contents entry
#   FRONT_MATTER   - Title page, copyright, editorial info
#   BACK_MATTER    - Bibliography, addresses, study programmes
#   PAGE_BREAK     - Page break marker
#   SKIP           - Non-content elements (containers, formatting spans)

CSS_CLASS_MAP = {
    # ── CHAPTER TITLES (Level 1 / h1) ──────────────────────────
    'Chapter-title-TOC-Level-1':            ('CHAPTER_TITLE', 1),
    'Chapter-title-TOC-Level-1-no-new-page':('CHAPTER_TITLE', 1),
    
    # ── SECTION HEADINGS (Level 2 / h2) ───────────────────────
    'Chapter-title-TOC-Level-2':            ('SECTION_HEAD', 2),
    'Text-heading-1':                       ('SECTION_HEAD', 2),
    'Sadhana-heading-2':                    ('SECTION_HEAD', 2),
    
    # ── SUBSECTIONS (Level 3 / h3) ────────────────────────────
    'Sadhana-heading-3':                    ('SUBSECTION', 3),
    'Sadhana-heading-3-centre':             ('SUBSECTION', 3),
    'Text-heading-3-italics-brackets':      ('SUBSECTION', 3),
    
    # ── SUBSUBSECTIONS (Level 4 / h4) ─────────────────────────
    'Sadhana-heading-4':                    ('SUBSUBSECTION', 4),
    
    # ── BODY TEXT ──────────────────────────────────────────────
    'Text-1st-para':                        ('BODY_FIRST', None),
    'Text-1st-para-space-below':            ('BODY_FIRST', None),
    'Text-1st-para-full-space-below':       ('BODY_FIRST', None),
    'Text-2nd-para':                        ('BODY', None),
    'Text-2nd-para-space-below':            ('BODY', None),
    'Text-2nd-para-full-space-below':       ('BODY', None),
    
    # ── VERSE / POETRY QUOTATIONS ─────────────────────────────
    'Text-quotation-verse-begin':           ('VERSE', None),
    'Text-quotation-verse-middle':          ('VERSE', None),
    'Text-quotation-verse-last':            ('VERSE', None),
    'Text-quotation-verse-single':          ('VERSE', None),
    'Text-quotation-verse-last-full-space-below': ('VERSE', None),
    'Text-quotation-verse-italic-begin':    ('VERSE', None),
    'Text-quotation-verse-italic-middle':   ('VERSE', None),
    'Text-quotation-verse-italic-last':     ('VERSE', None),
    'Text-quotation-verse-italic-single':   ('VERSE', None),
    'Text-quotation-verse-italic-last-full-space-below': ('VERSE', None),
    
    # ── PROSE QUOTATIONS ──────────────────────────────────────
    'Text-quotation-prose-single':          ('QUOTE_PROSE', None),
    'Text-quotation-prose-single-last':     ('QUOTE_PROSE', None),
    'Text-quotation-prose-single-full-space-below': ('QUOTE_PROSE', None),
    
    # ── LIST ITEMS ────────────────────────────────────────────
    'Text-list-1-to-9-begin':              ('LIST_ITEM', None),
    'Text-list-1-to-9-middle':             ('LIST_ITEM', None),
    'Text-list-1-to-9-last':               ('LIST_ITEM', None),
    'Text-list-1-to-9-last-full-space-below': ('LIST_ITEM', None),
    'Text-list-1-to-9-with-parenthesis-begin':  ('LIST_ITEM', None),
    'Text-list-1-to-9-with-parenthesis-middle': ('LIST_ITEM', None),
    'Text-list-1-to-9-with-parenthesis-last':   ('LIST_ITEM', None),
    'Text-list-1-to-9-with-parenthesis-last-full-space-below': ('LIST_ITEM', None),
    'Text-list-10-to-99-with-parenthesis-middle': ('LIST_ITEM', None),
    'Text-list-10-to-99-with-parenthesis-last':   ('LIST_ITEM', None),
    
    # ── SADHANA / MEDITATION INSTRUCTIONS ─────────────────────
    'Sadhana-instruction-first-para':       ('INSTRUCTION', None),
    'Sadhana-instruction-middle-para':      ('INSTRUCTION', None),
    'Sadhana-instruction-last-para':        ('INSTRUCTION', None),
    'Sadhana-instruction-full-space-below': ('INSTRUCTION', None),
    
    # ── SADHANA PROSE (liturgical text) ───────────────────────
    'Sadhana-prose-space-below':            ('SADHANA_PROSE', None),
    'Sadhana-prose-full-space-below':       ('SADHANA_PROSE', None),
    
    # ── MANTRA ────────────────────────────────────────────────
    'Mantra':                               ('MANTRA', None),
    
    # ── CHART TITLES ──────────────────────────────────────────
    'Text-title-2':                         ('CHART_TITLE', None),
    
    # ── SIGNATURES ────────────────────────────────────────────
    'Text-Signature-Author-2-lines-plus':   ('BODY', None),
    
    # ── IMAGE CAPTIONS ────────────────────────────────────────
    'Image-caption':                        ('IMAGE_CAPTION', None),
    'Illustration-entry':                   ('IMAGE_CAPTION', None),
    
    # ── TABLE OF CONTENTS ─────────────────────────────────────
    'TOC-Body-Text':                        ('TOC', None),
    'Sadhana-content-level-2':              ('TOC', None),
    'Sadhana-content-level-2-space-after':  ('TOC', None),
    'Sadhana-content-level-3':              ('TOC', None),
    
    # ── GLOSSARY ──────────────────────────────────────────────
    'Glossary-entry':                       ('GLOSSARY', None),
    
    # ── INDEX ─────────────────────────────────────────────────
    'Index-Section-Head':                   ('INDEX', None),
    'Index-level-1':                        ('INDEX', None),
    'Index-level-1-space-below':            ('INDEX', None),
    'Index-level-2':                        ('INDEX', None),
    'Index-level-2-space-below':            ('INDEX', None),
    'Index-level-3':                        ('INDEX', None),
    'Index-level-3-space-below':            ('INDEX', None),
    'Index-Text':                           ('INDEX', None),
    
    # ── FRONT MATTER ──────────────────────────────────────────
    'Book-Title-inside':                    ('FRONT_MATTER', None),
    'Book-title-title-page':                ('FRONT_MATTER', None),
    'Chapter-subtitle':                     ('FRONT_MATTER', None),
    'Author':                               ('FRONT_MATTER', None),
    'Also-by':                              ('FRONT_MATTER', None),
    'Also-by-other-title':                  ('FRONT_MATTER', None),
    'Editorial-info':                       ('FRONT_MATTER', None),
    'Editorial-info-no-space-below':        ('FRONT_MATTER', None),
    'Editorial-info-no-space-below-1st':    ('FRONT_MATTER', None),
    'Editorial-info-addresses':             ('FRONT_MATTER', None),
    'copyright-Text':                       ('FRONT_MATTER', None),
    'Tharpa-logo':                          ('FRONT_MATTER', None),
    'Profits':                              ('FRONT_MATTER', None),
    'Profits-top':                          ('FRONT_MATTER', None),
    
    # ── BACK MATTER ───────────────────────────────────────────
    'Biblio-entry-2-lines':                 ('BACK_MATTER', None),
    'Biblio-entry-2-lines-space-below':     ('BACK_MATTER', None),
    'Biblio-entry-2-lines-full-space-below':('BACK_MATTER', None),
    'Biblio-text':                          ('BACK_MATTER', None),
    'Biblio-title-entry':                   ('BACK_MATTER', None),
    'Tharpa-Offices-Text':                  ('BACK_MATTER', None),
    'Tharpa-Offices-Address':               ('BACK_MATTER', None),
    'Tharpa-Offices-Address-Title':         ('BACK_MATTER', None),
    'Tharpa-Offices-Address-Last':          ('BACK_MATTER', None),
    'Tharpa-logo-country':                  ('BACK_MATTER', None),
    'Study-program-text':                   ('BACK_MATTER', None),
    'Study-program-text-2nd-para':          ('BACK_MATTER', None),
    'Study-program-text-2nd-para-full-space-below': ('BACK_MATTER', None),
    'Study-program-text-space-below':       ('BACK_MATTER', None),
    'Study-program-text-full-space-below':  ('BACK_MATTER', None),
    'Study-program-center-address':         ('BACK_MATTER', None),
    'Study-Program-List-middle':            ('BACK_MATTER', None),
    'Study-Program-List-last':              ('BACK_MATTER', None),
    
    # ── PAGE BREAKS ───────────────────────────────────────────
    'ebook-page-break':                     ('PAGE_BREAK', None),
    
    # ── SKIP (non-content: containers, inline formatting) ─────
    # These are <div> containers or <span> formatting, not <p> content
    'Basic-Text-Frame':                     ('SKIP', None),
    'line-drawings':                        ('SKIP', None),
    'line-drawings-no-page-before-and-after': ('SKIP', None),
    '_idGenObjectStyle-Disabled':           ('SKIP', None),
    'Author-Colour-Image':                  ('SKIP', None),
    'tharpa-logo-image':                    ('SKIP', None),
    'nkt-logo-back-image':                  ('SKIP', None),
    'temple-image':                         ('SKIP', None),
    # Inline formatting spans
    'Italic':                               ('SKIP', None),
    'Bold':                                 ('SKIP', None),
    'Bold-Italic':                          ('SKIP', None),
    'Small-Caps':                           ('SKIP', None),
    'Small-Caps-Italic':                    ('SKIP', None),
    'Normal':                               ('SKIP', None),
    '_1---Hyperlink':                       ('SKIP', None),
    '_1---Small-Caps---BOLD':               ('SKIP', None),
}

# Roles that count as "teaching content" (for chapter content detection)
TEACHING_ROLES = {
    'BODY', 'BODY_FIRST', 'VERSE', 'QUOTE_PROSE', 'LIST_ITEM',
    'INSTRUCTION', 'SADHANA_PROSE', 'MANTRA', 'CHART_TITLE'
}

# Roles that are headings
HEADING_ROLES = {'CHAPTER_TITLE', 'SECTION_HEAD', 'SUBSECTION', 'SUBSUBSECTION'}

print(f"✓ CSS class map defined: {len(CSS_CLASS_MAP)} classes mapped")
print(f"  Heading roles: {len([v for v in CSS_CLASS_MAP.values() if v[0] in HEADING_ROLES])}")
print(f"  Content roles: {len([v for v in CSS_CLASS_MAP.values() if v[0] in TEACHING_ROLES])}")
print(f"  Other roles:   {len([v for v in CSS_CLASS_MAP.values() if v[0] not in HEADING_ROLES and v[0] not in TEACHING_ROLES])}")

✓ CSS class map defined: 111 classes mapped
  Heading roles: 9
  Content roles: 38
  Other roles:   64


## Data Structures

Enhanced schema adds `structural_role`, `heading_level`, and `parent_section_id` to each paragraph, plus `sections` list within each chapter.

In [3]:
@dataclass
class Paragraph:
    paragraph_id: str
    text: str
    chapter_index: int
    paragraph_index: int
    char_start: int
    char_end: int
    page_number: int
    sentence_count: int
    structural_role: str          # NEW: BODY, VERSE, INSTRUCTION, etc.
    css_class: str                # NEW: original CSS class name
    heading_level: Optional[int]  # NEW: 1-4 for headings, None for body
    parent_section: Optional[str] # NEW: heading text of parent section
    epub_section_idx: int         # NEW: which EPUB section this came from
    
    def to_dict(self):
        return {
            'paragraph_id': self.paragraph_id,
            'text': self.text,
            'chapter_index': self.chapter_index,
            'paragraph_index': self.paragraph_index,
            'char_start': self.char_start,
            'char_end': self.char_end,
            'page_number': self.page_number,
            'sentence_count': self.sentence_count,
            'structural_role': self.structural_role,
            'css_class': self.css_class,
            'heading_level': self.heading_level,
            'parent_section': self.parent_section,
            'epub_section_idx': self.epub_section_idx,
        }


@dataclass
class Section:
    """A section heading within a chapter (h2/h3/h4)."""
    heading_text: str
    heading_level: int
    paragraph_index: int  # index of this heading in the chapter's paragraph list
    
    def to_dict(self):
        return {
            'heading_text': self.heading_text,
            'heading_level': self.heading_level,
            'paragraph_index': self.paragraph_index,
        }


@dataclass
class Chapter:
    chapter_index: int
    chapter_title: Optional[str]
    epub_section_indices: List[int]  # which EPUB sections make up this chapter
    pages: List[int]
    sections: List[Section]         # NEW: section headings within this chapter
    paragraphs: List[Paragraph]
    
    def to_dict(self):
        return {
            'chapter_index': self.chapter_index,
            'chapter_title': self.chapter_title,
            'epub_section_indices': self.epub_section_indices,
            'pages': list(sorted(set(self.pages))),
            'sections': [s.to_dict() for s in self.sections],
            'paragraphs': [p.to_dict() for p in self.paragraphs],
        }

print("✓ Data structures defined")

✓ Data structures defined


## Pass 1: Extract All Paragraphs with Structural Metadata

Walk every EPUB section, classify every `<p>` tag by its CSS class, and record the structural role. This is the raw extraction — hierarchy building happens in Pass 2.

In [4]:
def classify_paragraph(p_tag):
    """
    Given a BeautifulSoup <p> tag, return (role, heading_level, css_class).
    Uses the CSS_CLASS_MAP. Unknown classes default to BODY.
    """
    classes = p_tag.get('class', [])
    
    if not classes:
        # No class attribute — treat as body text
        return 'BODY', None, 'no-class'
    
    # Use first class (Tharpa only uses single classes per <p>)
    css_class = classes[0]
    
    if css_class in CSS_CLASS_MAP:
        role, level = CSS_CLASS_MAP[css_class]
        return role, level, css_class
    else:
        # Unknown class — default to BODY, log it
        return 'BODY', None, css_class


def count_sentences(text):
    """Simple sentence count."""
    if not text:
        return 0
    # Split on sentence-ending punctuation followed by space or end
    sentences = re.split(r'[.!?]+(?:\s|$)', text)
    return max(1, len([s for s in sentences if s.strip()]))


# ── Extract all paragraphs ──────────────────────────────────
book = epub.read_epub(EPUB_PATH)
epub_items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

all_paragraphs = []  # List of (epub_section_idx, role, level, css_class, text)
unknown_classes = set()
role_counts = defaultdict(int)
page_number = 1  # Track page numbers from page break markers

print(f"Extracting from {len(epub_items)} EPUB sections...")

for section_idx, item in enumerate(epub_items):
    content = item.get_content()
    soup = BeautifulSoup(content, 'html.parser')
    
    for p_tag in soup.find_all('p'):
        text = p_tag.get_text(strip=True)
        if not text:
            continue
        
        role, level, css_class = classify_paragraph(p_tag)
        
        # Track unknown classes
        if css_class not in CSS_CLASS_MAP and css_class != 'no-class':
            unknown_classes.add(css_class)
        
        # Track page breaks
        if role == 'PAGE_BREAK':
            page_number += 1
            continue  # Don't store "page break" as a paragraph
        
        # Skip non-content elements
        if role == 'SKIP':
            continue
        
        role_counts[role] += 1
        all_paragraphs.append({
            'epub_section_idx': section_idx,
            'role': role,
            'heading_level': level,
            'css_class': css_class,
            'text': text,
            'page_number': page_number,
        })

print(f"\n✓ Extracted {len(all_paragraphs)} paragraphs")
print(f"\nParagraphs by role:")
for role, count in sorted(role_counts.items(), key=lambda x: -x[1]):
    print(f"  {role:20s}: {count:4d}")

if unknown_classes:
    print(f"\n⚠️  Unknown CSS classes (defaulted to BODY): {unknown_classes}")
else:
    print(f"\n✓ All CSS classes recognized")

print(f"\nPage count (from page breaks): {page_number}")

Extracting from 89 EPUB sections...

✓ Extracted 3481 paragraphs

Paragraphs by role:
  INDEX               : 1108
  BODY                :  850
  LIST_ITEM           :  403
  BACK_MATTER         :  250
  BODY_FIRST          :  248
  GLOSSARY            :  153
  SECTION_HEAD        :  113
  VERSE               :   91
  FRONT_MATTER        :   60
  IMAGE_CAPTION       :   53
  TOC                 :   37
  CHAPTER_TITLE       :   32
  SUBSECTION          :   28
  QUOTE_PROSE         :   20
  INSTRUCTION         :   16
  SUBSUBSECTION       :    9
  SADHANA_PROSE       :    6
  CHART_TITLE         :    4

⚠️  Unknown CSS classes (defaulted to BODY): {'Sadhana-verse-x3-full-space-below', 'Sadhana-verse-last-full-space-below', 'Sadhana-colophon', 'Sadhana-verse-Liberating-Prayer-middle', 'Sadhana-Mantra-indent-space-below', 'Sadhana-verse-Liberating-Prayer-last', 'Sadhana-Mantra-indent-full-space-below', 'Sadhana-Mantra-no-indent', 'Sadhana-Mantra-indent', 'Sadhana-verse-last', 'Sadhana-vers

## Pass 2: Build Chapter/Section Hierarchy

Walk the extracted paragraphs in order. When we hit a `CHAPTER_TITLE`, start a new chapter. When we hit a `SECTION_HEAD`/`SUBSECTION`/`SUBSUBSECTION`, record a section within the current chapter. Everything else gets appended to the current chapter's paragraph list.

This collapses the 89 EPUB sections into the real chapter structure.

In [5]:
chapters = []
current_chapter = None
global_para_idx = 0
char_position = 0  # running character offset

for para_data in all_paragraphs:
    role = para_data['role']
    level = para_data['heading_level']
    text = para_data['text']
    
    # ── New chapter? ────────────────────────────────────────
    if role == 'CHAPTER_TITLE':
        # Save previous chapter
        if current_chapter is not None:
            chapters.append(current_chapter)
        
        current_chapter = Chapter(
            chapter_index=len(chapters),
            chapter_title=text,
            epub_section_indices=[para_data['epub_section_idx']],
            pages=[],
            sections=[],
            paragraphs=[],
        )
        # Don't store the chapter title as a paragraph — it's metadata
        continue
    
    # ── Handle content before first chapter title ───────────
    # (cover, "also by", title page, etc.)
    if current_chapter is None:
        # Create a "Front Matter" pseudo-chapter
        current_chapter = Chapter(
            chapter_index=0,
            chapter_title="Front Matter",
            epub_section_indices=[],
            pages=[],
            sections=[],
            paragraphs=[],
        )
    
    # ── Track which EPUB sections contribute to this chapter ──
    if para_data['epub_section_idx'] not in current_chapter.epub_section_indices:
        current_chapter.epub_section_indices.append(para_data['epub_section_idx'])
    
    # ── Determine parent section heading ────────────────────
    parent_section = None
    if current_chapter.sections:
        parent_section = current_chapter.sections[-1].heading_text
    
    # ── Record section headings ─────────────────────────────
    if role in HEADING_ROLES and role != 'CHAPTER_TITLE':
        section = Section(
            heading_text=text,
            heading_level=level,
            paragraph_index=len(current_chapter.paragraphs),
        )
        current_chapter.sections.append(section)
        parent_section = text  # This heading is its own parent
    
    # ── Create paragraph ────────────────────────────────────
    para = Paragraph(
        paragraph_id=f"clb_ch{current_chapter.chapter_index}_para{len(current_chapter.paragraphs)}",
        text=text,
        chapter_index=current_chapter.chapter_index,
        paragraph_index=len(current_chapter.paragraphs),
        char_start=char_position,
        char_end=char_position + len(text),
        page_number=para_data['page_number'],
        sentence_count=count_sentences(text),
        structural_role=role,
        css_class=para_data['css_class'],
        heading_level=level,
        parent_section=parent_section,
        epub_section_idx=para_data['epub_section_idx'],
    )
    
    current_chapter.paragraphs.append(para)
    current_chapter.pages.append(para_data['page_number'])
    char_position = para.char_end + 2  # +2 for paragraph separator
    global_para_idx += 1

# Don't forget the last chapter
if current_chapter is not None:
    chapters.append(current_chapter)

# ── Summary ──
total_paragraphs = sum(len(ch.paragraphs) for ch in chapters)
total_sections = sum(len(ch.sections) for ch in chapters)

print(f"✓ Built {len(chapters)} chapters")
print(f"  {total_sections} section headings")
print(f"  {total_paragraphs} total paragraphs")
print(f"\nChapter listing:")
print(f"{'─' * 80}")
print(f"{'Idx':>4s}  {'Title':50s}  {'Paras':>5s}  {'Sects':>5s}  {'Pages':>10s}")
print(f"{'─' * 80}")
for ch in chapters:
    title = (ch.chapter_title or "(no title)")[:50]
    page_range = f"{min(ch.pages)}-{max(ch.pages)}" if ch.pages else "?"
    print(f"{ch.chapter_index:4d}  {title:50s}  {len(ch.paragraphs):5d}  {len(ch.sections):5d}  {page_range:>10s}")

✓ Built 33 chapters
  150 section headings
  3449 total paragraphs

Chapter listing:
────────────────────────────────────────────────────────────────────────────────
 Idx  Title                                               Paras  Sects       Pages
────────────────────────────────────────────────────────────────────────────────
   0  Front Matter                                            1      0         1-1
   1  About the Author                                       57      0         1-1
   2  Contents                                               32      0         1-1
   3  Illustrations                                          26      0         1-1
   4  Foreword                                               25      0         2-2
   5  Acknowledgements                                        7      0         2-2
   6  Preface                                                 8      0         2-3
   7  Introduction and Preliminaries                        117      6         3-6
   8  

## Inspect: Section Headings per Chapter

Verify the heading hierarchy looks correct — especially important for teaching chapters.

In [6]:
# Show section headings for chapters that have them
for ch in chapters:
    if not ch.sections:
        continue
    print(f"\n{'═' * 70}")
    print(f"Chapter {ch.chapter_index}: {ch.chapter_title}")
    print(f"{'─' * 70}")
    for s in ch.sections:
        indent = "  " * (s.heading_level - 1) if s.heading_level else "  "
        print(f"  {indent}[h{s.heading_level}] {s.heading_text}")


══════════════════════════════════════════════════════════════════════
Chapter 7: Introduction and Preliminaries
──────────────────────────────────────────────────────────────────────
    [h2] AN INTRODUCTION TO THE GENERAL PATHS
    [h2] THE SOURCE OF THE LINEAGE FROM WHICH THESE INSTRUCTIONS ARE DERIVED
    [h2] THE ACTUAL EXPLANATION OF THE INSTRUCTIONS POSSESSING THIS LINEAGE
    [h2] THE PRELIMINARY PRACTICES
    [h2] THE COMMONPRELIMINARY PRACTICES
    [h2] THE UNCOMMONPRELIMINARY PRACTICES

══════════════════════════════════════════════════════════════════════
Chapter 8: Channels, Winds and Drops
──────────────────────────────────────────────────────────────────────
    [h2] THE ACTUAL PRACTICE
    [h2] HOW TO PRACTISE THE MAHAMUDRA THAT IS THE UNION OF BLISS AND EMPTINESS
    [h2] AN EXPLANATION OF THE METHOD FOR GENERATING THE OBJECT-POSSESSOR, SPONTANEOUS GREAT BLISS
    [h2] PENETRATING THE PRECISE POINTS OF ONE’S OWN BODY
    [h2] IDENTIFYING THE TEN DOORS THROUGH WHICH TH

## Inspect: Content Type Distribution

How many paragraphs of each structural role? Important for understanding the teaching content mix.

In [7]:
from collections import Counter

role_dist = Counter()
for ch in chapters:
    for p in ch.paragraphs:
        role_dist[p.structural_role] += 1

print(f"{'Role':25s}  {'Count':>6s}  {'%':>6s}")
print(f"{'─' * 42}")
total = sum(role_dist.values())
for role, count in role_dist.most_common():
    pct = 100 * count / total
    marker = " ← teaching" if role in TEACHING_ROLES else ""
    marker = " ← heading" if role in HEADING_ROLES else marker
    print(f"{role:25s}  {count:6d}  {pct:5.1f}%{marker}")
print(f"{'─' * 42}")
print(f"{'TOTAL':25s}  {total:6d}")

teaching_count = sum(v for k, v in role_dist.items() if k in TEACHING_ROLES)
print(f"\nTeaching content: {teaching_count} paragraphs ({100*teaching_count/total:.1f}%)")

Role                        Count       %
──────────────────────────────────────────
INDEX                        1108   32.1%
BODY                          850   24.6% ← teaching
LIST_ITEM                     403   11.7% ← teaching
BACK_MATTER                   250    7.2%
BODY_FIRST                    248    7.2% ← teaching
GLOSSARY                      153    4.4%
SECTION_HEAD                  113    3.3% ← heading
VERSE                          91    2.6% ← teaching
FRONT_MATTER                   60    1.7%
IMAGE_CAPTION                  53    1.5%
TOC                            37    1.1%
SUBSECTION                     28    0.8% ← heading
QUOTE_PROSE                    20    0.6% ← teaching
INSTRUCTION                    16    0.5% ← teaching
SUBSUBSECTION                   9    0.3% ← heading
SADHANA_PROSE                   6    0.2% ← teaching
CHART_TITLE                     4    0.1% ← teaching
──────────────────────────────────────────
TOTAL                        3449

Teach

## "Show Me" Tests

Verify specific content is correctly placed — these are the tests from the pipeline reference.

In [8]:
print("=" * 70)
print('"SHOW ME" TESTS')
print("=" * 70)

# Test 1: Show me a chapter title
print("\n1. Show me chapter titles (should NOT be null):")
for ch in chapters[:8]:
    print(f"   Ch {ch.chapter_index}: \"{ch.chapter_title}\"")

# Test 2: Show me 3 paragraphs from a teaching chapter
teaching_chs = [ch for ch in chapters if len(ch.paragraphs) > 20]
if teaching_chs:
    sample_ch = teaching_chs[0]
    print(f"\n2. Show me 3 paragraphs from Chapter {sample_ch.chapter_index} (\"{sample_ch.chapter_title}\")")
    body_paras = [p for p in sample_ch.paragraphs if p.structural_role in ('BODY', 'BODY_FIRST')]
    for p in body_paras[:3]:
        preview = p.text[:120] + "..." if len(p.text) > 120 else p.text
        print(f"   [{p.structural_role}] {preview}")

# Test 3: Show me an instruction paragraph
instructions = []
for ch in chapters:
    for p in ch.paragraphs:
        if p.structural_role == 'INSTRUCTION':
            instructions.append((ch.chapter_title, p))
if instructions:
    print(f"\n3. Show me an INSTRUCTION paragraph:")
    ch_title, p = instructions[0]
    preview = p.text[:150] + "..." if len(p.text) > 150 else p.text
    print(f"   Chapter: \"{ch_title}\"")
    print(f"   Text: {preview}")
else:
    print(f"\n3. No INSTRUCTION paragraphs found (may be in sadhana appendix)")

# Test 4: Show me a verse quotation
verses = []
for ch in chapters:
    for p in ch.paragraphs:
        if p.structural_role == 'VERSE':
            verses.append((ch.chapter_title, p))
if verses:
    print(f"\n4. Show me a VERSE quotation:")
    ch_title, p = verses[0]
    print(f"   Chapter: \"{ch_title}\"")
    print(f"   Text: \"{p.text}\"")

# Test 5: Show me section headings for a teaching chapter
if teaching_chs:
    ch = teaching_chs[0]
    print(f"\n5. Show me section headings in Chapter {ch.chapter_index} (\"{ch.chapter_title}\"):")
    for s in ch.sections[:5]:
        print(f"   [h{s.heading_level}] {s.heading_text}")

"SHOW ME" TESTS

1. Show me chapter titles (should NOT be null):
   Ch 0: "Front Matter"
   Ch 1: "About the Author"
   Ch 2: "Contents"
   Ch 3: "Illustrations"
   Ch 4: "Foreword"
   Ch 5: "Acknowledgements"
   Ch 6: "Preface"
   Ch 7: "Introduction and Preliminaries"

2. Show me 3 paragraphs from Chapter 1 ("About the Author")
   [BODY_FIRST] Geshe Kelsang Gyatso is a fully accomplished meditation master and internationally renowned teacher of Buddhism who has ...
   [BODY] Find our more atwww.tharpa.com

3. Show me an INSTRUCTION paragraph:
   Chapter: "The Yoga of Buddha Heruka"
   Text: At this point we imagine that wisdom being Je Tsongkhapa together with his retinue dissolves into the assembly of Guru Sumati Buddha Heruka, and they ...

4. Show me a VERSE quotation:
   Chapter: "Introduction and Preliminaries"
   Text: "By depending upon this boat-like human form,"

5. Show me section headings in Chapter 1 ("About the Author"):


## Save to JSON

In [9]:
output = {
    'book_id': 'Clear_Light_of_Bliss',
    'book_title': 'Clear Light of Bliss',
    'creator': 'Geshe Kelsang Gyatso',
    'extraction_version': '2.0_css_class_aware',
    'total_chapters': len(chapters),
    'total_sections': total_sections,
    'total_paragraphs': total_paragraphs,
    'total_pages': max(p.page_number for ch in chapters for p in ch.paragraphs) if chapters else 0,
    'structural_roles': dict(role_dist.most_common()),
    'css_class_map_version': 'phase0_2026-02',
    'chapters': [ch.to_dict() for ch in chapters],
}

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

file_size = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
print(f"✓ Saved to {OUTPUT_FILE}")
print(f"  File size: {file_size:.1f} MB")
print(f"  {output['total_chapters']} chapters")
print(f"  {output['total_sections']} sections")
print(f"  {output['total_paragraphs']} paragraphs")
print(f"  {output['total_pages']} pages")

✓ Saved to 06_document_structure_layer1.json
  File size: 2.1 MB
  33 chapters
  150 sections
  3449 paragraphs
  49 pages


## 🚦 Validation Gate 1A + 3A: Structure Quality

In [10]:
print("=" * 70)
print("🚦 VALIDATION GATE 1A + 3A: Document Structure Quality")
print("=" * 70)

# Reload to verify the saved file
with open(OUTPUT_FILE) as f:
    saved = json.load(f)

checks = []

# Check 1: Chapter titles exist
null_titles = sum(1 for ch in saved['chapters'] if ch.get('chapter_title') is None)
total_ch = len(saved['chapters'])
check1 = null_titles == 0
checks.append(('All chapters have titles', check1, 
                f"{total_ch - null_titles}/{total_ch} have titles"))

# Check 2: Realistic chapter count (not 88 EPUB sections)
check2 = 10 <= total_ch <= 40  # Should be ~30-35 with front/back matter
checks.append(('Realistic chapter count (10-40)', check2,
                f"{total_ch} chapters"))

# Check 3: Paragraphs exist
check3 = saved['total_paragraphs'] > 1000
checks.append(('Paragraph count > 1000', check3,
                f"{saved['total_paragraphs']} paragraphs"))

# Check 4: Section headings exist
check4 = saved['total_sections'] > 10
checks.append(('Section headings > 10', check4,
                f"{saved['total_sections']} sections"))

# Check 5: Heading levels exist in paragraphs
has_levels = any(
    p.get('heading_level') is not None
    for ch in saved['chapters']
    for p in ch.get('paragraphs', [])
)
checks.append(('Heading levels populated', has_levels, ''))

# Check 6: Structural roles populated
has_roles = all(
    p.get('structural_role') is not None
    for ch in saved['chapters']
    for p in ch.get('paragraphs', [])
)
checks.append(('Structural roles populated', has_roles, ''))

# Check 7: No "page break" text in paragraphs
page_break_paras = sum(
    1 for ch in saved['chapters']
    for p in ch.get('paragraphs', [])
    if p.get('text', '').strip().lower() == 'page break'
)
check7 = page_break_paras == 0
checks.append(('No "page break" text in paragraphs', check7,
                f"{page_break_paras} found"))

all_pass = True
for desc, passed, detail in checks:
    status = "✓" if passed else "✗"
    if not passed:
        all_pass = False
    detail_str = f" ({detail})" if detail else ""
    print(f"  {status} {desc}{detail_str}")

if all_pass:
    print(f"\n  ✅ ALL CHECKS PASSED")
    print(f"  Document structure is ready for Phase 2 NLP + Phase 3 Neo4j!")
else:
    print(f"\n  ⚠️  SOME CHECKS FAILED — review output above")

🚦 VALIDATION GATE 1A + 3A: Document Structure Quality
  ✓ All chapters have titles (33/33 have titles)
  ✓ Realistic chapter count (10-40) (33 chapters)
  ✓ Paragraph count > 1000 (3449 paragraphs)
  ✓ Section headings > 10 (150 sections)
  ✓ Heading levels populated
  ✓ Structural roles populated
  ✓ No "page break" text in paragraphs (0 found)

  ✅ ALL CHECKS PASSED
  Document structure is ready for Phase 2 NLP + Phase 3 Neo4j!


## Before/After Comparison

Compare the new extraction against the old `06_document_structure_layer1.json` to confirm improvements.

In [11]:
# Try to load the old file for comparison
old_path = "06_document_structure_layer1_OLD.json"
if not os.path.exists(old_path):
    # Check if current file is the old one (before overwrite)
    print("ℹ️  No old file found for comparison.")
    print("  Tip: Before running this notebook, rename the old file:")
    print(f"    mv 06_document_structure_layer1.json {old_path}")
else:
    with open(old_path) as f:
        old = json.load(f)
    
    print(f"{'Metric':<35s}  {'OLD':>10s}  {'NEW':>10s}  {'Status':>8s}")
    print(f"{'─' * 70}")
    
    old_null = sum(1 for ch in old['chapters'] if ch.get('chapter_title') is None)
    new_null = sum(1 for ch in saved['chapters'] if ch.get('chapter_title') is None)
    print(f"{'Null chapter titles':<35s}  {old_null:>10d}  {new_null:>10d}  {'✓ FIXED' if new_null < old_null else '✗'}")
    
    print(f"{'Total chapters':<35s}  {old['total_chapters']:>10d}  {saved['total_chapters']:>10d}  {'✓' if saved['total_chapters'] < old['total_chapters'] else '?'}")
    print(f"{'Total paragraphs':<35s}  {old['total_paragraphs']:>10d}  {saved['total_paragraphs']:>10d}")
    
    old_sections = sum(len(ch.get('sections', [])) for ch in old['chapters'])
    new_sections = saved['total_sections']
    print(f"{'Section headings':<35s}  {old_sections:>10d}  {new_sections:>10d}  {'✓ NEW' if new_sections > old_sections else ''}")
    
    has_roles_old = any(p.get('structural_role') for ch in old['chapters'] for p in ch.get('paragraphs', []))
    print(f"{'Has structural_role field':<35s}  {'No':>10s}  {'Yes':>10s}  {'✓ NEW'}")
    print(f"{'Has heading_level field':<35s}  {'No':>10s}  {'Yes':>10s}  {'✓ NEW'}")
    print(f"{'Has parent_section field':<35s}  {'No':>10s}  {'Yes':>10s}  {'✓ NEW'}")

ℹ️  No old file found for comparison.
  Tip: Before running this notebook, rename the old file:
    mv 06_document_structure_layer1.json 06_document_structure_layer1_OLD.json
